# Code for Cahn-Hilliard phase separation without mechanical coupling.

## 2D phase separation study.


Degrees of freedom: 
  - scalar chemical potential: we use normalized  mu = mu/RT 
  - species concentration:  we use normalized  c= Omega*cmat 
  
### Units:
- Length: um
- Mass: kg
- Time: s
- Amount of substance: pmol
- Temperature: K
- Mass density: kg/um^3
- Force: uN
- Stress: MPa
- Energy: pJ
- Species concentration: pmol/um^3
- Chemical potential: pJ/pmol
- Molar volume: um^3/pmol
- Species diffusivity: um^2/s
- Boltzmann Constant: 1.38E-11 pJ/K
- Gas constant: 8.314  pJ/(pmol K)

### By
  Eric Stewart      and      Lallit Anand
ericstew@mit.edu            anand@mit.edu

October 2023

Modified for FenicsX by Jorge Nin
jorgenin@mit.edu

In [1]:
import numpy as np


from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log
from dolfinx.fem import Constant, dirichletbc, Function, FunctionSpace, Expression
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import VTXWriter
import ufl
from ufl import (
    TestFunction,
    TrialFunction,
    Identity,
    grad,
    det,
    div,
    dev,
    inv,
    tr,
    sqrt,
    conditional,
    gt,
    dx,
    inner,
    derivative,
    dot,
    ln,
    split,
    tanh,
    as_tensor,
    as_vector,
    ge
)
from datetime import datetime
from dolfinx.plot import vtk_mesh

import pyvista

pyvista.set_jupyter_backend("client")
## Define temporal parameters
import random

# DEFINE GEOMETRY

In [2]:
problemName = "Canh Hillard"

# Square edge length
L0 = 0.8  # 800 nm box, after Di Leo et al. (2014)

# Number of elements along each side
N = 100

# Create square mesh
domain = mesh.create_rectangle(MPI.COMM_WORLD, [(0, 0), (L0, L0)], [N, N])

## Visualize the geometry

In [3]:
#sysctl -a|grep "\.shm"

plotter = pyvista.Plotter()
vtkdata = vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
actor = plotter.add_mesh(grid, show_edges=True)
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:55463/index.html?ui=P_0x2a4e09310_0&reconnect=auto' style='width: …

# Periodicity

In [4]:
def inside(self, x, on_boundary):
    # return True if on left or bottom boundary AND NOT
    # on one of the two corners (0, L0) and (L0, 0)
    return bool(
        (np.isclose(x[0], 0) or np.isclose(x[1], 0))
        and (
            not (
                (np.isclose(x[0], 0) and np.isclose(x[1], L0))
                or (np.isclose(x[0], L0) and np.isclose(x[1], 0))
            )
        )
        and on_boundary
    )


def map(self, x, y):
    if np.isclose(x[0], L0) and np.isclose(x[1], L0):
        y[0] = x[0] - L0
        y[1] = x[1] - L0
    elif np.isclose(x[0], L0):
        y[0] = x[0] - L0
        y[1] = x[1]
    else:  # np.isclose(x[1], L0)
        y[0] = x[0]
        y[1] = x[1] - L0

# Material Parameters

In [5]:
# Material parameters after Di Leo et al. (2014)
#
Omega = Constant(domain, 4.05)  # Molar volume, um^3/pmol
D = Constant(domain, 1e-2)  # Diffusivity, um^2/s
chi = Constant(domain, 3.0)  # Phase parameter, (-)
cMax = Constant(domain, 2.29e-2)  # Saturation concentration, pmol/um^3
lam = Constant(domain, 5.5749e-1)  # Interface parameter, (pJ/pmol) um^2
theta0 = Constant(domain, 298.0)  # Reference temperature, K
R_gas = Constant(domain, 8.3145)  # Gas constant, pJ/(pmol K)

RT = R_gas * theta0

# Simulation Time Control

In [6]:
t = 0.0  # initialization of time
Ttot = 4  # total simulation time
dt = 0.01  # Initial time step size, here we will use adaptive time-stepping

# Function Spaces

In [7]:
U2 = ufl.VectorElement("Lagrange", domain.ufl_cell(), 2)  # For displacement
P1 = ufl.FiniteElement("Lagrange", domain.ufl_cell(), 1)  # For  pressure
# For  normalized chemical potential
# and  normalized species concentration

TH = ufl.MixedElement([P1, P1])  # Taylor-Hood style mixed element
ME = FunctionSpace(domain, TH)  # Total space for all DOFs
x = ufl.SpatialCoordinate(domain)

In [8]:
# Define trial functions
w = Function(ME)
mu, c = split(w)  # chemical potential  mu, concentration c

# A copy of functions to store values in the previous step
w_old = Function(ME)
mu_old, c_old = split(w_old)

# Define test functions
w_test = TestFunction(ME)
mu_test, c_test = split(w_test)

# Define trial functions needed for automatic differentiation
dw = TrialFunction(ME)

In [9]:
np.random.seed(2 + MPI.COMM_WORLD.Get_rank())

# Some quick definitions to init random values
V, _ = ME.sub(0).collapse()
cBar_rand = Function(V)
cBar_rand.interpolate(
    lambda x: 0.63 + 0.05 * (0.5 - np.random.rand(x.shape[1]))
)  # Create a nice random value we can use
fc_rand = RT * (
    ln(cBar_rand / (1 - cBar_rand)) + chi * (1 - 2 * cBar_rand)
)  # use that relation to initate the two different sub expressions


# Concentration Expressoin
concentration = Expression(Omega * cMax * cBar_rand, ME.sub(1).element.interpolation_points())
w.sub(1).interpolate( concentration )

chemical_potential = Expression(fc_rand / RT, ME.sub(0).element.interpolation_points())
w.sub(0).interpolate(chemical_potential)

w_old.x.array[:] = w.x.array



# Subroutines

In [10]:
"""
For 2D plane strain:
"""


# Gradient of vector field u
def pe_grad_vector(u):
    grad_u = grad(u)
    return as_tensor(
        [[grad_u[0, 0], grad_u[0, 1], 0], [grad_u[1, 0], grad_u[1, 1], 0], [0, 0, 0]]
    )


# Gradient of scalar field y
# (just need an extra zero for dimensions to work out)
def pe_grad_scalar(y):
    grad_y = grad(y)
    return as_vector([grad_y[0], grad_y[1], 0.0])


# ------------------------------------------------------------------------------
# Species flux
def Flux_calc(mu, c):
    #
    cBar = c / (Omega * cMax)  # normalized concentration
    #
    Mob = (D * c) / (Omega * RT) * (1 - cBar)
    #
    Jmat = -RT * Mob * pe_grad_scalar(mu)
    return Jmat


# Calculate the f^c term
def fc_calc(mu, c):
    #
    cBar = c / (Omega * cMax)  # normalized concentration
    #
    fc = RT * (ln(cBar / (1 - cBar)) + chi * (1 - 2 * cBar))
    #
    return fc


# Eigenvalue decomposition of a 2D tensor
def eigs(T):
    # Compute eigenvalues
    lambda1_0 = (
        T[0, 0] / 2
        + T[1, 1] / 2
        - sqrt(
            T[0, 0] ** 2 - 2 * T[0, 0] * T[1, 1] + 4 * T[0, 1] * T[1, 0] + T[1, 1] ** 2
        )
        / 2
    )
    lambda2_0 = (
        T[0, 0] / 2
        + T[1, 1] / 2
        + sqrt(
            T[0, 0] ** 2 - 2 * T[0, 0] * T[1, 1] + 4 * T[0, 1] * T[1, 0] + T[1, 1] ** 2
        )
        / 2
    )

    # Compute eigenvectors
    v11 = -T[1, 1] + lambda1_0
    v12 = T[1, 0]

    v21 = -T[1, 1] + lambda2_0
    v22 = T[1, 0]

    vec1_0 = as_vector([v11, v12])
    vec2_0 = as_vector([v21, v22])
    
    # Normalize eigenvectors
    vec1_0 = vec1_0 / sqrt(dot(vec1_0, vec1_0))
    vec2_0 = vec2_0 / sqrt(dot(vec2_0, vec2_0))

    # Reorder eigenvectors and eigenvalues
    vec1 = conditional(ge(lambda1_0, lambda2_0), vec1_0, vec2_0)
    vec2 = conditional(ge(lambda1_0, lambda2_0), vec2_0, vec1_0)

    lambda1 = conditional(ge(lambda1_0, lambda2_0), lambda1_0, lambda2_0)
    lambda2 = conditional(ge(lambda1_0, lambda2_0), lambda2_0, lambda1_0)

    return lambda1, lambda2, vec1, vec2

# Kinematics and constitutive relations

In [11]:
# Calculate the normalized concentration cBar
cBar = c/(Omega*cMax) # normalized concentration

# Calculate the Species flux
Jmat = Flux_calc(mu, c)

# Calculate the f^c term 
fc = fc_calc(mu, c)


# Weak Forms

In [12]:
# Residuals:
# Res_0: Balance of mass   (test fxn: mu)
# Res_1: chemical potential (test fxn: c)
dx = ufl.dx(metadata={"quadrature_degree": 4})

# Time step field, constant within body
dk = Constant(domain,dt)

# The weak form for the mass balance of mobile species    
Res_0 = dot((c - c_old)/dk, mu_test)*dx \
        -  Omega*dot(Jmat , pe_grad_scalar(mu_test) )*dx      

# The weak form for the concentration
Res_1 = dot(mu - fc/RT, c_test)*dx \
        -  dot( (lam/RT)*pe_grad_scalar(cBar), pe_grad_scalar(c_test))*dx
        
# Total weak form
Res = Res_0 + Res_1

# Automatic differentiation tangent:
a = derivative(Res, w, dw)

# Setup Output Files

In [13]:
U1 = ufl.VectorElement("Lagrange", domain.ufl_cell(), 1)
V2 = fem.FunctionSpace(domain, U1)#Vector function space
V1 = fem.FunctionSpace(domain, P1)#Scalar function space

mu_vis = Function(V1)
mu_vis.name = "mu"
mu_expr = Expression(mu,V1.element.interpolation_points())

c_vis = Function(V1)
c_vis.name = "c"
c_expr = Expression(c,V1.element.interpolation_points())

vtk = VTXWriter(domain.comm,"results/"+problemName+".bp", [mu_vis,c_vis], engine="BP4" )

def interp_and_save(t, file):
   
    mu_vis.interpolate(mu_expr)
    c_vis.interpolate(c_expr)

    file.write(t)


# Boundary Conditions

In [14]:
# Nothing! Just let the system evolve on its own.
bcs = []

# SETUP NONLINEAR PROBLEM

In [15]:
import os
step = "Swell"
jit_options ={"cffi_extra_compile_args":["-O3","-ffast-math"]}

problem = NonlinearProblem(Res, w, bcs, a)


solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-8
solver.atol = 1e-8
solver.max_it = 50
solver.report = True


ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_max_it"] = 30
opts[f"{option_prefix}ksp_type"] = "gmres"
opts[f"{option_prefix}pc_type"] = "ksp"
ksp.setFromOptions()

startTime = datetime.now()
print("------------------------------------")
print("Simulation Start")
print("------------------------------------")

step = "Evolve"

#if os.path.exists("results/"+problemName+".bp"):
#    os.remove("results/"+problemName+".xdmf")
#    os.remove("results/"+problemName+".h5")

#vtk.write_mesh(domain)
t = 0.0

interp_and_save(t, vtk)
ii = 0
bisection_count = 0
while t < Ttot:
    # increment time
    t += float(dk) 
    # increment counter
    ii +=1
    

    # Solve the problem
    try:
        (iter, converged) = solver.solve(w)
        
        w.x.scatter_forward()
        
        
        
        w_old.x.array[:] = w.x.array
        
        interp_and_save(t, vtk)
        if ii % 1 == 0:
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print("Step: {} |   Increment: {} | Iterations: {}".format(step, ii, iter))
            print("Simulation Time: {} s | dt: {} s".format(round(t, 2), round(dt, 3)))
            print()
        
        if iter <= 2:
            dt = 1.5 * dt
            dk.value = dt
        # If the newton solver takes 5 or more iterations,
        # decrease the time step by a factor of 2:
        elif iter >= 5:
            dt = dt / 2
            dk.value =dt

        #Reset Biseciton Counter
        bisection_count = 0
        
    except: # Break the loop if solver fails
        bisection_count += 1
        
        if bisection_count > 5:
            print("Error: Too many bisections")
            break
        
        print( "Error Halfing Time Step")
        t = t - float(dk)
        dt = dt / 2
        dk.value = dt
        print(f"New Time Step: {dt}")
        w.x.array[:] = w_old.x.array
        

#End Analysis
vtk.close()
endTime = datetime.now()
print("------------------------------------")
print("Simulation End")
print("------------------------------------")
print("Total Time: {}".format(endTime - startTime))
print("------------------------------------")



    
    

------------------------------------
Simulation Start
------------------------------------
Step: Evolve |   Increment: 1 | Iterations: 4
Simulation Time: 0.01 s | dt: 0.01 s

Step: Evolve |   Increment: 2 | Iterations: 3
Simulation Time: 0.02 s | dt: 0.01 s

Step: Evolve |   Increment: 3 | Iterations: 3
Simulation Time: 0.03 s | dt: 0.01 s

Step: Evolve |   Increment: 4 | Iterations: 3
Simulation Time: 0.04 s | dt: 0.01 s

Step: Evolve |   Increment: 5 | Iterations: 3
Simulation Time: 0.05 s | dt: 0.01 s

Step: Evolve |   Increment: 6 | Iterations: 3
Simulation Time: 0.06 s | dt: 0.01 s

Step: Evolve |   Increment: 7 | Iterations: 3
Simulation Time: 0.07 s | dt: 0.01 s

Step: Evolve |   Increment: 8 | Iterations: 3
Simulation Time: 0.08 s | dt: 0.01 s

Step: Evolve |   Increment: 9 | Iterations: 3
Simulation Time: 0.09 s | dt: 0.01 s

Step: Evolve |   Increment: 10 | Iterations: 3
Simulation Time: 0.1 s | dt: 0.01 s

Step: Evolve |   Increment: 11 | Iterations: 3
Simulation Time: 0.11 

: 